In [12]:
run mysetup.py

Setup complete.


# Movement

## Data Generation

## Solving Optimization 

In [14]:
timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir    = pathlib.Path.cwd() / "output" / timestamp
plots_root  = base_dir / "result"
base_dir.mkdir(parents=True, exist_ok=True)
plots_root.mkdir(parents=True, exist_ok=True)


In [15]:
"""
small_demo.py  –  demo of the housing-movement optimisation
author: ChatGPT (OpenAI)   date: 2025-05-28
"""
import itertools, random, numpy as np, pandas as pd
from pathlib import Path
from pulp import (LpProblem, LpMinimize, LpBinary, LpVariable,
                  lpSum, PULP_CBC_CMD)

# ------------------------------------------------------------------
# 1 ▸ generate a toy instance (10 PICs, 4 houses, 3 sets, 5 groups) --
# ------------------------------------------------------------------
random.seed(42);  np.random.seed(42)

PIC   = [f'P{i+1}' for i in range(10)]
HOUSE = [f'H{h+1}' for h in range(4)]
SETS  = [f'SET{s+1}' for s in range(3)]
L     = ['MO', 'NO_MO', 'MIN', 'MED', 'MAX']

# (a) PIC profile  --------------------------------------------------
violence = np.random.choice(['MIN','MED','MAX'], size=len(PIC))
is_mo    = np.random.choice([0,1], size=len(PIC))
df_L = pd.DataFrame.from_records([
    dict(PIC_ID=pid,
         **{g:int(g=='MO')*mo + int(g=='NO_MO')*(1-mo) +
              int(g==v) for g in L})
    for pid, v, mo in zip(PIC, violence, is_mo)
])

# (b) initial assignment  ------------------------------------------
df_init = pd.DataFrame({
    'PIC_ID'        : PIC,
    'HOUSE_ID_INIT' : np.random.choice(HOUSE, size=len(PIC))
})

# (c) set profile  --------------------------------------------------
membership = np.zeros((len(PIC), len(SETS)), dtype=int)
for i in range(len(PIC)):
    if random.random() < .8:
        membership[i, random.randrange(len(SETS))] = 1
df_S = pd.DataFrame(membership, columns=SETS).assign(PIC_ID=PIC)

# (d) random separation-order & SRG conflicts -----------------------
df_so = pd.DataFrame(random.sample(
            list(itertools.combinations(PIC, 2)), 3),
            columns=['PIC_ID1','PIC_ID2'])

df_sc = pd.DataFrame(random.sample(
            list(itertools.combinations(SETS, 2)), 2),
            columns=['SET1','SET2'])

# (e) physical housing & type menus ---------------------------------
df_ph = pd.DataFrame({
    'HOUSE_ID'   : HOUSE,
    'TYPE_BUILD' : np.random.choice(['DORM','CELL'], size=len(HOUSE)),
    'CAP'        : np.random.randint(3,5, size=len(HOUSE))
})

TYPES = ['MO_MIN','MO_MED','NO_MO_MIN','NO_MO_MED','NO_MO_MAX']
df_tb = pd.DataFrame([(h,t) for h in HOUSE for t in TYPES],
                     columns=['HOUSE_ID','TYPE_ID'])

w = {                     # group-permission matrix w_{tℓ}
    'MO_MIN'   : dict(MO=1, NO_MO=0, MIN=1, MED=0, MAX=0),
    'MO_MED'   : dict(MO=1, NO_MO=0, MIN=0, MED=1, MAX=0),
    'NO_MO_MIN': dict(MO=0, NO_MO=1, MIN=1, MED=0, MAX=0),
    'NO_MO_MED': dict(MO=0, NO_MO=1, MIN=0, MED=1, MAX=0),
    'NO_MO_MAX': dict(MO=0, NO_MO=1, MIN=0, MED=0, MAX=1),
}
df_tbl = (pd.DataFrame(w).T
          .reset_index(names='TYPE_ID')
          .reindex(columns=['TYPE_ID',*L]))

# ------------------------------------------------------------------
# 2 ▸ build the MILP  (exactly the formulation in the note)   ------
# ------------------------------------------------------------------
cap      = df_ph.set_index('HOUSE_ID')['CAP'].to_dict()
init_h   = df_init.set_index('PIC_ID')['HOUSE_ID_INIT'].to_dict()
x_is     = {(r.PIC_ID,s):r[s] for _,r in df_S.iterrows() for s in SETS}
y_il     = {(r.PIC_ID,l):r[l] for _,r in df_L.iterrows() for l in L}
w_tl     = {(r.TYPE_ID,l):r[l] for _,r in df_tbl.iterrows() for l in L}
T_of_h   = {h:df_tb[df_tb.HOUSE_ID==h]['TYPE_ID'].tolist() for h in HOUSE}

prob = LpProblem("Housing_Min_Move", LpMinimize)

a = LpVariable.dicts('a', (PIC, HOUSE), 0, 1, LpBinary)   # assignment
b = LpVariable.dicts('b', (HOUSE, TYPES), 0, 1, LpBinary) # house type
u = LpVariable.dicts('u', HOUSE, 0, 1, LpBinary)          # house used
m = LpVariable.dicts('m', PIC, 0, 1, LpBinary)            # moved?

# (i) each PIC in exactly one house
for i in PIC:
    prob += lpSum(a[i][h] for h in HOUSE) == 1

# (ii) one type per house
for h in HOUSE:
    prob += lpSum(b[h][t] for t in T_of_h[h]) == 1

# (iii) link u_h
for h in HOUSE:
    for i in PIC:
        prob += a[i][h] <= u[h]

# (iv) capacity
for h in HOUSE:
    prob += lpSum(a[i][h] for i in PIC) <= cap[h]

# (v) movement flag  m_i = 1 – a_{i,init(i)}
for i in PIC:
    prob += m[i] + a[i][init_h[i]] == 1

# (vi) separation-order
for _,row in df_so.iterrows():
    p1,p2 = row.PIC_ID1, row.PIC_ID2
    for h in HOUSE:
        prob += a[p1][h] + a[p2][h] <= 1

# (vii) SRG conflicts
for _,row in df_sc.iterrows():
    s1,s2 = row.SET1, row.SET2
    for h in HOUSE:
        prob += (
            lpSum(a[i][h]*x_is[i,s1] for i in PIC) +
            lpSum(a[i][h]*x_is[i,s2] for i in PIC) <= 1)

# (viii) type-based limits
for h in HOUSE:
    for l in L:
        lhs = lpSum(a[i][h]*y_il[i,l] for i in PIC)
        rhs = cap[h]*lpSum(b[h][t]*w_tl[t,l] for t in T_of_h[h])
        prob += lhs <= rhs

# objective = moves  + 0.1 × houses-used
prob += lpSum(m[i] for i in PIC) + 0.1*lpSum(u[h] for h in HOUSE)

# ------------------------------------------------------------------
# 3 ▸ solve & export report   --------------------------------------
# ------------------------------------------------------------------
prob.solve(PULP_CBC_CMD(msg=False))

moves = pd.DataFrame([
    dict(PIC_ID=i,
         HOUSE_ID_INIT = init_h[i],
         HOUSE_ID_NEW  = next(h for h in HOUSE if a[i][h].value()>0.5))
    for i in PIC
])
moves['MOVED_FLAG'] = (moves.HOUSE_ID_INIT != moves.HOUSE_ID_NEW).astype(int)

summary = pd.DataFrame([{
    'TOTAL_MOVES' : int(moves.MOVED_FLAG.sum()),
    'HOUSES_USED' : int(sum(u[h].value() for h in HOUSE))
}])





ModuleNotFoundError: No module named 'pulp'

In [ ]:
pip install  pulp 

Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/pulp/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pulp/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pulp (from versions: none)
ERROR: No matching distribution found for pulp


In [ ]:
path_result = base_dir / 'output_moves.xlsx'
with pd.ExcelWriter(path_result, engine='xlsxwriter') as writer:
    moves.to_excel(writer,   sheet_name='Moves',    index=False)
    summary.to_excel(writer, sheet_name='Summary',  index=False)

print("✓ optimisation complete →", out.resolve())